# 🏠 서울 아파트 실거래가 예측 모델

## 모델 개요
- **알고리즘**: RandomForest Regressor
- **피처 수**: 21개 (수치형 15개 + 범주형 6개)
- **Validation 방식**: 시계열 분할 (Time-based Split)

## 시계열 Validation
부동산 가격 예측은 **미래 시점의 가격**을 예측하는 문제, 따라서 Validation도 실제 예측 상황과 동일하게 **과거 → 미래** 순서로 분할.

```
일반적인 Random Split의 문제:
  Train/Valid에 2017~2023 데이터가 무작위로 섞임
  → 2023년 데이터로 학습하고 2020년 데이터로 검증하는 상황 발생
  → Validation 점수가 실제 성능을 반영하지 못함

시계열 Split:
  Train: 2017.01 ~ 2023.03 (과거 데이터로 학습)
  Valid: 2023.04 ~ 2023.06 (Test 직전 시기로 검증)
  Test:  2023.07 ~ 2023.09 (실제 예측 대상)
  → Validation이 실제 Test 상황을 모사함
```

## 1. Library Import

In [18]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestRegressor
from scipy.spatial import cKDTree
import warnings
warnings.filterwarnings('ignore')

from lab_core.util.path import raw_data_dir, ext_data_dir, out_dir

print("라이브러리 로드 완료")

라이브러리 로드 완료


## 2. 데이터 로드

In [19]:
# 경로 설정
train_path = raw_data_dir('train.csv')
test_path = raw_data_dir('test.csv')
subway_path = raw_data_dir('subway_feature.csv')
bus_path = raw_data_dir('bus_feature.csv')
coord_path = ext_data_dir('coord.csv')  # 카카오 API로 생성한 좌표 데이터

# 데이터 로드
train = pd.read_csv(train_path)
test = pd.read_csv(test_path)
subway = pd.read_csv(subway_path)
bus = pd.read_csv(bus_path)
coord = pd.read_csv(coord_path)

print(f"Train: {train.shape}")
print(f"Test: {test.shape}")
print(f"지하철역: {len(subway)}개")
print(f"버스정류장: {len(bus):,}개")
print(f"좌표 데이터: {len(coord):,}개")

Train: (1118822, 52)
Test: (9272, 51)
지하철역: 768개
버스정류장: 12,584개
좌표 데이터: 8,477개


In [20]:
# 데이터 기간 확인
print("[Train 기간]")
print(f"  계약년월: {train['계약년월'].min()} ~ {train['계약년월'].max()}")

print("\n[Test 기간]")
print(f"  계약년월: {test['계약년월'].min()} ~ {test['계약년월'].max()}")

[Train 기간]
  계약년월: 200701 ~ 202306

[Test 기간]
  계약년월: 202307 ~ 202309


## 3. 전처리 함수 정의

In [21]:
def haversine_distance(lat1, lon1, lat2, lon2):
    """위경도 기반 거리 계산 (km)
    
    Haversine 공식을 사용하여 두 지점 간의 대원거리(great-circle distance)를 계산합니다.
    지구를 완전한 구로 가정합니다.
    
    Parameters:
        lat1, lon1: 첫 번째 지점의 위도, 경도
        lat2, lon2: 두 번째 지점의 위도, 경도
    
    Returns:
        두 지점 간의 거리 (km)
    """
    R = 6371  # 지구 반지름 (km)
    lat1, lon1, lat2, lon2 = map(np.radians, [lat1, lon1, lat2, lon2])
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    a = np.sin(dlat/2)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon/2)**2
    c = 2 * np.arcsin(np.sqrt(a))
    return R * c

In [22]:
def fill_missing_coords(train_df, test_df, coord_df):
    """좌표 결측치 채우기 (coord.csv 활용)
    
    카카오 API로 미리 수집한 좌표 데이터(coord.csv)를 사용하여
    Train/Test의 좌표 결측치를 채웁니다.
    
    매핑 키: '구 + 도로명' (예: '강남구 테헤란로 123')
    
    Parameters:
        train_df: 학습 데이터프레임
        test_df: 테스트 데이터프레임
        coord_df: 좌표 데이터프레임 (매핑키, 좌표X, 좌표Y 컬럼 필요)
    
    Returns:
        좌표가 채워진 (train, test) 튜플
    """
    train = train_df.copy()
    test = test_df.copy()
    
    # 매핑 키 생성 (구 + 도로명)
    train['구'] = train['시군구'].str.split().str[1]
    test['구'] = test['시군구'].str.split().str[1]
    train['매핑키'] = train['구'] + ' ' + train['도로명'].fillna('')
    test['매핑키'] = test['구'] + ' ' + test['도로명'].fillna('')
    
    # 좌표 딕셔너리 생성 (NaN 제외)
    coord_valid = coord_df[coord_df['좌표X'].notna()]
    coord_dict = dict(zip(coord_valid['매핑키'], zip(coord_valid['좌표X'], coord_valid['좌표Y'])))
    print(f"좌표 딕셔너리 크기: {len(coord_dict):,}개")
    
    # Train 좌표 결측치 채우기
    train_before = train['좌표X'].isna().sum()
    mask = train['좌표X'].isna()
    train.loc[mask, '좌표X'] = train.loc[mask, '매핑키'].map(lambda x: coord_dict.get(x, (np.nan, np.nan))[0])
    train.loc[mask, '좌표Y'] = train.loc[mask, '매핑키'].map(lambda x: coord_dict.get(x, (np.nan, np.nan))[1])
    train_after = train['좌표X'].isna().sum()
    
    # Test 좌표 결측치 채우기
    test_before = test['좌표X'].isna().sum()
    mask = test['좌표X'].isna()
    test.loc[mask, '좌표X'] = test.loc[mask, '매핑키'].map(lambda x: coord_dict.get(x, (np.nan, np.nan))[0])
    test.loc[mask, '좌표Y'] = test.loc[mask, '매핑키'].map(lambda x: coord_dict.get(x, (np.nan, np.nan))[1])
    test_after = test['좌표X'].isna().sum()
    
    # 임시 컬럼 제거
    train = train.drop(columns=['매핑키'], errors='ignore')
    test = test.drop(columns=['매핑키'], errors='ignore')
    
    # 결과 출력
    print(f"\n[좌표 결측치 채우기 결과]")
    print(f"  Train: {train_before:,} → {train_after:,} (채움: {train_before - train_after:,})")
    print(f"  Test: {test_before:,} → {test_after:,} (채움: {test_before - test_after:,})")
    print(f"\n  Train 커버율: {train['좌표X'].notna().mean()*100:.1f}%")
    print(f"  Test 커버율: {test['좌표X'].notna().mean()*100:.1f}%")
    

    # 남은 결측치는 구 평균 좌표로 채우기
    combined = pd.concat([train, test], ignore_index=True)
    gu_mean = combined[combined['좌표X'].notna()].groupby('구')[['좌표X', '좌표Y']].mean().to_dict('index')
    
    for df in [train, test]:
        mask = df['좌표X'].isna()
        for idx in df[mask].index:
            gu = df.loc[idx, '구']
            if gu in gu_mean:
                df.loc[idx, '좌표X'] = gu_mean[gu]['좌표X']
                df.loc[idx, '좌표Y'] = gu_mean[gu]['좌표Y']


    return train, test

In [23]:
def create_basic_features(df):
    """기본 피처 생성
    
    원본 데이터에서 파생되는 기본 피처들을 생성합니다:
    - 시군구에서 구, 동 추출
    - 계약년월에서 년, 월, 분기 추출
    - 건물 나이 및 신축 여부 계산
    - 강남3구 여부 플래그
    - 면적/층 구간 범주화
    
    Parameters:
        df: 입력 데이터프레임
    
    Returns:
        피처가 추가된 데이터프레임
    """
    df = df.copy()
    
    # 지역 정보 파싱 (구 컬럼이 없는 경우에만 생성)
    if '구' not in df.columns:
        df['구'] = df['시군구'].str.split().str[1]
    df['동'] = df['시군구'].str.split().str[2]
    
    # 시간 관련 피처
    df['계약년'] = df['계약년월'] // 100
    df['계약월'] = df['계약년월'] % 100
    df['계약분기'] = (df['계약월'] - 1) // 3 + 1
    
    # 건물 관련 피처
    df['건물나이'] = df['계약년'] - df['건축년도']
    df['신축여부'] = (df['건물나이'] <= 10).astype(int)  # 10년 이하면 신축
    
    # 강남권 여부 (부동산 프리미엄 지역)
    강남3구 = ['강남구', '서초구', '송파구']
    df['강남여부'] = df['구'].isin(강남3구).astype(int)
    
    # 면적 구간 범주화
    bins = [0, 40, 60, 85, 135, float('inf')]
    labels = ['초소형', '소형', '중형', '중대형', '대형']
    df['면적구간'] = pd.cut(df['전용면적(㎡)'], bins=bins, labels=labels).astype(str)
    
    # 층 구간 범주화
    bins = [0, 5, 10, 20, float('inf')]
    labels = ['저층', '중저층', '중층', '고층']
    df['층구간'] = pd.cut(df['층'], bins=bins, labels=labels).astype(str)
    
    return df

In [24]:
def add_transport_features(df, subway_df, bus_df):
    """교통 접근성 피처 추가
    
    KDTree를 사용하여 각 아파트에서 가장 가까운 지하철역/버스정류장까지의
    거리와 일정 반경 내 교통시설 개수를 계산합니다.
    
    생성되는 피처:
    - 지하철_최근접_거리: 가장 가까운 지하철역까지 거리 (km)
    - 버스_최근접_거리: 가장 가까운 버스정류장까지 거리 (km)
    - 지하철_500m_개수: 반경 500m 내 지하철역 수
    - 버스_300m_개수: 반경 300m 내 버스정류장 수
    
    Parameters:
        df: 아파트 데이터프레임 (좌표X, 좌표Y 필요)
        subway_df: 지하철역 데이터프레임
        bus_df: 버스정류장 데이터프레임
    
    Returns:
        교통 피처가 추가된 데이터프레임
    """
    df = df.copy()
    
    # KDTree 구축 (빠른 최근접 이웃 탐색을 위해)
    subway_coords = subway_df[['위도', '경도']].values
    subway_tree = cKDTree(subway_coords)
    bus_coords = bus_df[['Y좌표', 'X좌표']].values
    bus_tree = cKDTree(bus_coords)
    apt_coords = df[['좌표Y', '좌표X']].values
    
    # 최근접 교통시설 찾기
    subway_dist, subway_idx = subway_tree.query(apt_coords, k=1)
    bus_dist, bus_idx = bus_tree.query(apt_coords, k=1)
    
    # 실제 거리 계산 (Haversine 공식 사용)
    df['지하철_최근접_거리'] = [
        haversine_distance(apt_coords[i, 0], apt_coords[i, 1],
            subway_coords[subway_idx[i], 0], subway_coords[subway_idx[i], 1])
        for i in range(len(df))
    ]
    df['버스_최근접_거리'] = [
        haversine_distance(apt_coords[i, 0], apt_coords[i, 1],
            bus_coords[bus_idx[i], 0], bus_coords[bus_idx[i], 1])
        for i in range(len(df))
    ]
    
    # 반경 내 교통시설 개수 (r은 좌표 단위, 대략적인 값)
    subway_500m = subway_tree.query_ball_point(apt_coords, r=0.005)  # 약 500m
    df['지하철_500m_개수'] = [len(x) for x in subway_500m]
    bus_300m = bus_tree.query_ball_point(apt_coords, r=0.003)  # 약 300m
    df['버스_300m_개수'] = [len(x) for x in bus_300m]
    
    return df

## 4. 전처리 실행

In [25]:
%%time
print("="*60)
print("Step 1: 좌표 결측치 채우기 (coord.csv 활용)")
print("="*60)
train, test = fill_missing_coords(train, test, coord)

print("\n" + "="*60)
print("Step 2: 기본 피처 생성")
print("="*60)
train = create_basic_features(train)
test = create_basic_features(test)
print("기본 피처 생성 완료")

print("\n" + "="*60)
print("Step 3: 교통 피처 생성")
print("="*60)
train = add_transport_features(train, subway, bus)
test = add_transport_features(test, subway, bus)
print("교통 피처 생성 완료")

Step 1: 좌표 결측치 채우기 (coord.csv 활용)
좌표 딕셔너리 크기: 8,237개

[좌표 결측치 채우기 결과]
  Train: 869,670 → 21,715 (채움: 847,955)
  Test: 6,562 → 17 (채움: 6,545)

  Train 커버율: 98.1%
  Test 커버율: 99.8%

Step 2: 기본 피처 생성
기본 피처 생성 완료

Step 3: 교통 피처 생성
교통 피처 생성 완료
CPU times: user 23.7 s, sys: 3.07 s, total: 26.7 s
Wall time: 27.6 s


## 5. 피처 선택 및 인코딩

In [26]:
# 사용할 피처 목록 (총 21개)
feature_cols = [
    # 수치형 피처 (15개)
    '전용면적(㎡)', '층', '건축년도', '좌표X', '좌표Y',
    '계약년', '계약월', '건물나이', '신축여부', '강남여부', '계약분기',
    '지하철_최근접_거리', '버스_최근접_거리', '지하철_500m_개수', '버스_300m_개수',
    # 범주형 피처 (6개)
    '구', '동', '아파트명', '도로명', '면적구간', '층구간'
]

categorical_cols = ['구', '동', '아파트명', '도로명', '면적구간', '층구간']

print(f"총 피처 수: {len(feature_cols)}개")
print(f"  - 수치형: {len(feature_cols) - len(categorical_cols)}개")
print(f"  - 범주형: {len(categorical_cols)}개")

총 피처 수: 21개
  - 수치형: 15개
  - 범주형: 6개


In [27]:
# 범주형 피처 Label Encoding
# Train과 Test에 모두 등장하는 값들을 통합하여 인코딩
encoders = {}

for col in categorical_cols:
    le = LabelEncoder()
    train[col] = train[col].fillna('UNKNOWN').astype(str)
    test[col] = test[col].fillna('UNKNOWN').astype(str)
    # Train + Test의 모든 고유값으로 인코더 학습
    all_values = list(set(train[col].unique()) | set(test[col].unique()))
    le.fit(all_values)
    train[col] = le.transform(train[col])
    test[col] = le.transform(test[col])
    encoders[col] = le
    print(f"  {col}: {len(le.classes_)}개 클래스")

print("\n인코딩 완료")

  구: 25개 클래스
  동: 337개 클래스
  아파트명: 6550개 클래스
  도로명: 9245개 클래스
  면적구간: 5개 클래스
  층구간: 5개 클래스

인코딩 완료


## 6. 시계열 기반 Train/Validation 분할

부동산 가격 예측은 **미래 예측** 문제이므로, Validation도 시간 순서를 유지해야 합니다.

- **Train**: 2017.01 ~ 2023.03 (과거 데이터)
- **Validation**: 2023.04 ~ 2023.06 (Test 직전 시기)
- **Test**: 2023.07 ~ 2023.09 (실제 예측 대상)

In [28]:
# 시계열 기준 분할
train_mask = train['계약년월'] <= 202303
valid_mask = (train['계약년월'] >= 202304) & (train['계약년월'] <= 202306)

X_train = train.loc[train_mask, feature_cols]
y_train = train.loc[train_mask, 'target']

X_val = train.loc[valid_mask, feature_cols]
y_val = train.loc[valid_mask, 'target']

X_test = test[feature_cols]

print("="*60)
print("시계열 Validation 분할 결과")
print("="*60)
print(f"Train: {X_train.shape[0]:,}건 (~ 2023.03)")
print(f"Valid: {X_val.shape[0]:,}건 (2023.04 ~ 2023.06)")
print(f"Test:  {X_test.shape[0]:,}건 (2023.07 ~ 2023.09)")

# 기간 확인
print(f"\n[Train 기간] {train.loc[train_mask, '계약년월'].min()} ~ {train.loc[train_mask, '계약년월'].max()}")
print(f"[Valid 기간] {train.loc[valid_mask, '계약년월'].min()} ~ {train.loc[valid_mask, '계약년월'].max()}")
print(f"[Test 기간]  {test['계약년월'].min()} ~ {test['계약년월'].max()}")

시계열 Validation 분할 결과
Train: 1,108,190건 (~ 2023.03)
Valid: 10,632건 (2023.04 ~ 2023.06)
Test:  9,272건 (2023.07 ~ 2023.09)

[Train 기간] 200701 ~ 202303
[Valid 기간] 202304 ~ 202306
[Test 기간]  202307 ~ 202309


## 7. 모델 학습

In [29]:
%%time
# RandomForest 모델 학습
model = RandomForestRegressor(
    n_estimators=200,      # 트리 개수
    max_depth=20,          # 최대 깊이
    min_samples_split=5,   # 분할을 위한 최소 샘플 수
    random_state=42,       # 재현성을 위한 시드
    n_jobs=-1              # 모든 CPU 코어 사용
)

model.fit(X_train, y_train)

# Validation 성능 평가
val_pred = model.predict(X_val)
val_rmse = np.sqrt(mean_squared_error(y_val, val_pred))

print(f"\n[시계열 Validation 결과]")
print(f"Validation RMSE: {val_rmse:,.0f}")
print(f"\n※ 시계열 분할을 사용했으므로, 이 RMSE가 실제 리더보드 점수와 유사할 것으로 기대됩니다.")


[시계열 Validation 결과]
Validation RMSE: 17,200

※ 시계열 분할을 사용했으므로, 이 RMSE가 실제 리더보드 점수와 유사할 것으로 기대됩니다.
CPU times: user 1h 12min 22s, sys: 7.59 s, total: 1h 12min 30s
Wall time: 4min 38s


In [30]:
# 피처 중요도 확인
importance = pd.DataFrame({
    'feature': feature_cols,
    'importance': model.feature_importances_
}).sort_values('importance', ascending=False)

print("[피처 중요도 Top 15]")
for i, (_, row) in enumerate(importance.head(15).iterrows()):
    print(f"  {i+1:2}. {row['feature']}: {row['importance']:.4f}")

[피처 중요도 Top 15]
   1. 전용면적(㎡): 0.3883
   2. 계약년: 0.1870
   3. 강남여부: 0.1359
   4. 건축년도: 0.0652
   5. 좌표X: 0.0565
   6. 좌표Y: 0.0530
   7. 구: 0.0207
   8. 건물나이: 0.0159
   9. 도로명: 0.0156
  10. 지하철_최근접_거리: 0.0107
  11. 버스_300m_개수: 0.0092
  12. 아파트명: 0.0089
  13. 버스_최근접_거리: 0.0088
  14. 동: 0.0087
  15. 층: 0.0059


## 8. 전체 Train으로 재학습 후 예측

Validation으로 성능을 확인한 후, 최종 제출을 위해 전체 Train 데이터로 다시 학습합니다.

In [31]:
%%time
# 제출용: 전체 Train 데이터로 재학습
print("전체 Train 데이터로 재학습 중...")

X_full = train[feature_cols]
y_full = train['target']

model_final = RandomForestRegressor(
    n_estimators=200,
    max_depth=20,
    min_samples_split=5,
    random_state=42,
    n_jobs=-1
)

model_final.fit(X_full, y_full)
print(f"학습 완료: {X_full.shape[0]:,}건")

전체 Train 데이터로 재학습 중...
학습 완료: 1,118,822건
CPU times: user 1h 19min 44s, sys: 15.7 s, total: 1h 20min
Wall time: 5min 11s


In [32]:
# Test 데이터 예측
test_pred = model_final.predict(X_test)

print(f"예측값 범위: {test_pred.min():,.0f} ~ {test_pred.max():,.0f}")
print(f"예측값 평균: {test_pred.mean():,.0f}")

예측값 범위: 7,615 ~ 1,204,781
예측값 평균: 108,942


In [33]:
# 제출 파일 생성
import datetime

now = datetime.datetime.now()
date_time_str = now.strftime('%Y%m%d_%H%M%S')
file_name = f'output_{date_time_str}.csv'
file_path = out_dir("wjk/" + file_name)

submission = pd.DataFrame({
    'target': test_pred.astype(int)
})

submission.to_csv(file_path, index=False)
print(f"제출 파일 저장: {file_name}")
print(submission.head())

OSError: Cannot save file into a non-existent directory: '/home/hyjeo/Workspaces/upstage-ml-ai22-team2/outputs/wjk'

---

## 참고: Validation 방식에 따른 차이

| 방식 | Train 데이터 | Validation 데이터 | 특징 |
|------|-------------|-------------------|------|
| Random Split | 2017~2023 무작위 섞임 | 2017~2023 무작위 섞임 | 미래 데이터로 과거를 평가하는 경우 발생 |
| **시계열 Split** | **~2023.03** | **2023.04~06** | **실제 예측 상황과 동일한 구조** |

## 성능 개선 아이디어

이 모델을 기반으로 다음과 같은 방향으로 개선해볼 수 있습니다:

1. **피처 추가**: 학군 정보, 주변 상권, 공원 접근성 등
2. **앙상블**: LightGBM, XGBoost 등 다른 모델과 결합
3. **하이퍼파라미터 튜닝**: Optuna 등을 활용한 최적화
4. **피처 선택**: 중요도가 낮은 피처 제거 실험